In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule

Построим граф всех победных комбинаций выбранного игрока настольной игры Okiya.  
Комбинации поражения в граф включать не будем, для экономии памяти.  
У каждой комбинации на своем ходе, будет вес, соответствующий количеству победных комбинаций в направлении финального хода.  
Так мы сможем сортировать комбинации хода, для выбора наиболее богатой потенциальными победами комбинации.

В игре два игрока. У каждого игрока свой граф побед.  
Размер поля 4 на 4 клетки.  
Возможно 16 ходов, по 8 на каждого игрока.  
Первым ходит игрок 0, Вторым игрок 1.

### win_combination

Для определения победной комбинации удобно использовать побитовый оператор И.  
Например, десятичное 51865 это двоичное 1100101010011001. Если разрезать на 4 части получится комбинация:  
1100  
1010  
1001  
1001  
Вертикальная линия слева - победная комбинация.  
Побитовый И действует как оператор умножения, умножая 1й бит слева на 1й бит справа и так каждый бит.  
Например, 111&010 вернёт 010. Это свойство позволяет использовать оператор И как фильтр.  
Что бы определить есть ли полный ряд единицы в левой колонке, применим оператор И на нашем числе 1100101010011001 и шаблоне 1000100010001000.  
В десятичном представлении это будет записано так: 51865&34952  
Если результат будет равен 34952 значит ряд единиц есть. В противном случае нули затрут одну или несколько единиц в колонке и в результате будет уже другое число.

In [2]:
51865&32768

32768

Опишем условие для каждой строки, каждой колонки и для квадратных победных комбинаций.

In [3]:
ker = SourceModule("""
__device__ bool win_combination(int i)
{
    int win[17]={
    61440, // row 0
    3840,  // row 1
    240,   // row 2
    15,    // row 3
    34952, // col 0
    17476, // col 1
    8738,  // col 2
    4369,  // col 3
    52224, // quad 0
    26112, // quad 1
    13056, // quad 2
    3264,  // quad 3
    1632,  // quad 4
    816,   // quad 5
    204,   // quad 6
    102,   // quad 7
    51     // quad 8
    };
    for (int a=0;a<17;a++) if ((i&win[a])==win[a]) return(true);    
    return(false);
}

__global__ void last_turn(float *outvec)
{
    int ones = 0;
    int a = 0;
    
    // Идентификатор блока-потока
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    
    // Убедимся, что число единиц и нулей в комбинации одинаково
    for(a=0;a<16;a++) if ((((int)round(pow(2,a)))&i)>0) ones++;
    
    // Проверим, победна ли комбинация среди возможных
    if (ones==8 && win_combination(i)) outvec[i] = 1;
}
""")

### last_turn

In [4]:
[1 if i%2 else 0 for i in range(16)]

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

Последним ходит игрок 1. Определим все возможные, победные комбинации для текущего игрока на этапе последнего хода.  
Для поля 4 на 4 возможно 2**(4 * 4) что соответствует 65536 комбинаций.  
Используем 65536 потоков GPU. Каждый поток - уникальная комбинация поля.  
256 блоков в 256 сетках.
Каждая комбинация предствалена в виде int числа от 0 до 65536. Для представления в виде фишек на поле, переведем его в bin, разделим последовательность чисел на 4 части, разместив сверху вниз, например:  
0 1 0 1  
0 1 0 1  
0 1 0 1  
0 1 0 1  

In [5]:
combination_check_gpu = ker.get_function("last_turn")

combinations = np.zeros(256*256).astype(np.float32)
combinations_gpu_out = gpuarray.to_gpu(combinations)

%time combination_check_gpu( combinations_gpu_out, block=(256,1,1), grid=(256,1,1))

CPU times: user 731 µs, sys: 165 µs, total: 896 µs
Wall time: 907 µs


In [6]:
out_combinations = combinations_gpu_out.get()

Количество победных комбинаций:

In [7]:
np.count_nonzero(out_combinations)

6315

Посмотрим, на некоторые победные комбинации.

In [8]:
def bin_interpretation(i):
    clear_bin = bin(i)[2:]
    lead_zeros = ''.join(['0' for z in range(16-len(clear_bin))])
    return lead_zeros+clear_bin

In [9]:
np.where(out_combinations == 1)[0]

array([  255,   383,   447, ..., 65088, 65152, 65280])

In [10]:
for i in np.where(out_combinations == 1)[0][:3]:
    row = []
    i_bin = bin_interpretation(i)
    row.append(i_bin[:4])
    row.append(i_bin[4:8])
    row.append(i_bin[8:12])
    row.append(i_bin[12:])
    print('\n')
    for r in row:
        print(r)



0000
0000
1111
1111


0000
0001
0111
1111


0000
0001
1011
1111
